DynamoDb
---
O Amazon DynamoDB é um banco de dados de
valores-chave e documentos que oferece
desempenho em milissegundos de um dígito em
qualquer escala.

Documentacao: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html

In [1]:
#CASO ESTEJA NO COLAB, É IMPORTANTE INSTALAR O BOTO3
!pip install boto3

     |████████████████████████████████| 133kB 848kB/s eta 0:00:01
     |████████████████████████████████| 81kB 1.8MB/s eta 0:00:01
     |████████████████████████████████| 9.2MB 651kB/s eta 0:00:01
     |████████████████████████████████| 143kB 686kB/s eta 0:00:01


In [ ]:
import boto3

Para isso é necessario criar uma conexão com o serviço AWS. Precisamos autenticar utilizando 3 parametros:

1- aws_secret_access_key

2- aws_secret_key_id

3- aws_session_token

Esses parametros mudam todos os dias, precisamos atualizar sempre. Para acessar basta ir na AWS Account>Account Details>AWS CLI

https://labs.vocareum.com/main/main.php?m=editor&nav=1&asnid=14334&stepid=14335

Documentacao: https://boto3.amazonaws.com/v1/documentation/api/latest/index.html

In [4]:
#OS DADOS ABAIXO MUDAM TODOS OS DIAS A CADA 2 HORAS
access = '#######################'
secret = '#######################'
session ='#######################'

In [ ]:
#Quando passamos dois asteriscos na chamada de
#uma função, estamos indicando que os parametros estão dentro de um dicionario:
def multiplica(x,y):
    print(x*y)

multiplica(3,2)
# ou
param = {'y':3,'x':2}
multiplica(**param) 

Agora vamos criar um recurso do DynamoDb. Através desse recurso, poderemos criar, ler, manipular as tabelas do DynamoDB

In [ ]:
dynamodb = boto3.resource('dynamodb', aws_access_key_id= access,
                        aws_secret_access_key= secret,
                        aws_session_token= session
                       ,region_name = "us-east-1", use_ssl=False)

Agora vamos criar uma tabela passando os parametros em formar de dicionario, como funciona o DynamoDB

In [ ]:
table_name = 'pet_20200415' #Nome da tabela
#Parametros de criação
params = {
        'TableName': table_name, #nome da tabela
        #Esquema das chaves
        'KeySchema': [
            {'AttributeName': 'id', 'KeyType': 'HASH'}, # hash será a chave primaria
            {'AttributeName': 'nome', 'KeyType': 'RANGE'} #range, não obrigatória, formaria uma chave composta
        ],
        #Tipo de dados da chave
        'AttributeDefinitions': [ 
            {'AttributeName': 'id', 'AttributeType': 'N'}, #N de number
            {'AttributeName': 'nome', 'AttributeType': 'S'} #S de string
        ],
        #Configuração de escrita e leitura por segundo
        'ProvisionedThroughput': {
            'ReadCapacityUnits': 3,
            'WriteCapacityUnits': 3
        }
    }
    

Após criarmos os parametros para a nossa tabela, vamos passar para o create_table, para que a nossa tabela seja finalmente criada no DynamoDb

In [ ]:
dynamodb.create_table(**params)

Abaixo, vamos fazer alguns insert na nossa nova tabela no DynamoDB

In [ ]:
table = dynamodb.Table("pet_20200415")
table.put_item(
   Item={"id":1,
        'nome': 'Jane',
        'tipo': 'cachorro',
        'raça': 'vira lata'
    })
table.put_item(
   Item={"id":2,
        'nome': 'Larry',
        'tipo': 'lagarto',
        'escama': 'lisa',
         'chifre':'frontal'
    })
table.put_item(
   Item={"id":3,
        'nome': 'Marcel',
        'tipo': 'macaco',
        'especie': 'mandril',
         'cor':['verde oliva','cinza']
        })

Vamos criar mais uma tabela passando os parametros em formar de dicionario, como funciona o DynamoDB

In [ ]:
table_name = 'amigo_20200415' #Nome da tabela
#Parametros de criação
params = {
        'TableName': table_name, #nome da tabela
        #Esquema das chaves
        'KeySchema': [
            {'AttributeName': 'id', 'KeyType': 'HASH'}, # hash será a chave primaria
           
        ],
        #Tipo de dados da chave
        'AttributeDefinitions': [ 
            {'AttributeName': 'id', 'AttributeType': 'S'}, #String
        
        ],
        #Configuração de escrita e leitura por segundo
        'ProvisionedThroughput': {
            'ReadCapacityUnits': 3,
            'WriteCapacityUnits': 3
        }
    }
    

In [ ]:
dynamodb.create_table(**params)

Imagine que temos varios a nossa API baixada na nossa maquina e nela existem varios amigos para serem inseridos, mas nao consgeuimos subir um por um, seria muito trabalhaso, então, vamos fazer deste modo.

Vamo utilizar a biblioteca uuid que tem o mesmo objetivo do identity no SQL, ou seja, ela vai criar um ID unico

In [ ]:
import json, uuid

j = open("amigos.json")
amigos = json.load(j)
for amigo in amigos:
  id = uuid.uuid1()
  amigo["id"] = str(id)
  print(amigo)

#agora de fato vamos inserir na nossa tabela:
  table.put_item(Item = amigo)